In [ ]:
import numpy as np
import math
import seaborn as sns

In [ ]:
init_state = np.array([
    [0, 0, 0],
    [0, 0, 0],
    [0, 0, 0]
])

In [ ]:
class Game:
    def __init__(self, state, FIRST=1):
        self.state = state
        self.empty = self.make_empty(state)
        self.first_player = FIRST
        
    def make_empty(self, state):
        emp = []
        for i in range(3):
            for j in range(3):
                if state[i][j] == 0:
                    emp.append(3*i + j)
        
        return emp
    
    def is_lose(self):
        a = self.next_opp()
        
        for i in range(3):
            if self.state[i][0] == self.state[i][1] == self.state[i][2] != 0:
                return True
            elif self.state[0][i] == self.state[1][i] == self.state[2][i] != 0:
                return True
        if self.state[0][0] == self.state[1][1] == self.state[2][2] != 0:
            return True
        if self.state[0][2] == self.state[1][1] == self.state[2][0] != 0:
            return True
        return 0
    
    def is_draw(self):
        a = self.next_opp()
        if self.is_lose():
            return 0
        if np.all(self.state):
            return 1
        else:
            return 0
        
    def is_done(self):
        if self.is_lose() or self.is_draw():
            return 1
        else:
            return 0
        
        
    def update(self, target):
        state = self.state.copy()
        x, y = target//3, target%3
        a = self.next_opp()
        state[x][y] = a
        return Game(state)
    
    
    def next_opp(self):
        a = b = 0
        for i in range(len(self.state)):
            for j in range(len(self.state)):
                if self.state[i][j] == self.first_player:
                    a += 1
                elif self.state[i][j] != 0:
                    b += 1
                    
        if a == b:
            return self.first_player
        else:
            return 2 + min(0, 1-self.first_player)

In [ ]:
class Random:
    def action(self, game):
        return np.random.choice(game.empty)

In [ ]:
class Node:
    def __init__(self, game):
        self.game = game # 상태
        self.w = 0 # 보상 누계
        self.n = 0 # 시행 횟수
        self.child_nodes = None  # 자녀 노드 군
        
    def playout(self, game):
        if game.is_lose():
            return -1

        if game.is_draw():
            return 0

        return -self.playout(game.update(np.random.choice(game.empty)))

    # 국면 가치 계산
    def evaluate(self):
        # 게임 종료 시
        if self.game.is_done():
            # 승패 결과로 가치 취득
            value = -1 if self.game.is_lose() else 0 # 패배 시 -1, 무승부 시 0

            # 보상 누계와 시행 횟수 갱신
            self.w += value
            self.n += 1
            return value

        # 자녀 노드가 존재하지 않는 경우
        if not self.child_nodes:
            # 플레이아웃으로 가치 얻기
            value = self.playout(self.game)

            # 보상 누계와 시행 횟수 갱신
            self.w += value
            self.n += 1

            # 자녀 노드 전개
            if self.n == 10: 
                self.expand()
            return value

        # 자녀 노드가 존재하는 경우
        else:
            # UCB1이 가장 큰 자녀 노드를 평가해 가치 얻기
            value = -self.next_child_node().evaluate() 

            # 보상 누계와 시행 횟수 갱신
            self.w += value
            self.n += 1
            return value

    # 자녀 노드 전개
    def expand(self):
        self.child_nodes = []
        for action in self.game.empty:
            self.child_nodes.append(Node(self.game.update(action)))

    # UCB1이 가장 큰 자녀 노드 얻기
    def next_child_node(self):
         # 시행 횟수가 0인 자녀 노드 반환
        for child_node in self.child_nodes:
            if child_node.n == 0:
                return child_node

        # UCB1 계산
        t = 0
        for c in self.child_nodes:
            t += c.n
        ucb1_values = []
        for child_node in self.child_nodes:
            ucb1_values.append(-child_node.w/child_node.n+(2*math.log(t)/child_node.n)**0.5)

        # UCB1이 가장 큰 자녀 노드 반환
        return self.child_nodes[np.argmax(ucb1_values)]    

In [ ]:
class MCTS:
    def __init__(self):
        self.tree = None
    
    
    def action(self, game):
        n_steps = 100
        # 현재 국면의 노드 생성
        self.tree = Node(game)
        self.tree.expand()

        # 100회 시뮬레이션 실행
        for _ in range(n_steps):
            self.tree.evaluate()

        # 시행 횟수가 가장 큰 값을 갖는 행동 반환
        legal_actions = game.empty
        n_list = []
        for c in self.tree.child_nodes:
            n_list.append(c.n)
        return game.empty[np.argmax(n_list)]

                

In [ ]:
g = Game(init_state)
m = MCTS()
# 

In [ ]:
def play(game, m1, m2):
    global score
    while 1:
        a1 = m1.action(game)
        game = game.update(a1)
#         print(game.state)
        if game.is_lose():
            score[0] += 1
#             print(game.state)
            return 
        elif game.is_draw():
            score[2] += 1
#             print(game.state)
            return 

        a2 = m2.action(game)
        game = game.update(a2)
#         print(game.state)
        if game.is_lose():
            score[1] += 1
#             print(game.state)
            return 
        elif game.is_draw():
            score[2] += 1
#             print(game.state)
            return 
        

In [ ]:
game = Game(init_state)
m1 = Random()
m2 = MCTS()

In [ ]:
%%time
score = [0, 0, 0]
for _ in range(100):
#     print(_)
#     print(score)
    play(game, m2, m1)
print(score)

score = [0, 0, 0]
for _ in range(100):
    play(game, m1, m2)
#     print(score)
print(score)